In [2]:
!pip install Langchain

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
!pip install langchain pymupdf


  Obtaining dependency information for pymupdf from https://files.pythonhosted.org/packages/ea/fe/ff2bb633c0934ba43c36184b8ed025092e946994dc6b4c764a0079f0ab3c/PyMuPDF-1.24.13-cp39-abi3-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.4/18.4 MB 7.6 MB/s eta 0:00:0000:0100:01
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [5]:
pip install --upgrade langchain


  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/49/09/72630413a7ded27684e33392a0ff52ff1c8ea6749fee641319e75f82072b/langchain-0.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-core<0.4.0,>=0.3.15 from https://files.pythonhosted.org/packages/c6/bf/bce8f38fa59038e1d67fbc145eab86c00229e366956191a0a452ee24194e/langchain_core-0.3.18-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-text-splitters<0.4.0,>=0.3.0 from https://files.pythonhosted.org/packages/ee/c6/5ba25c8bad647e92a92b3066177ab10d78efbd16c0b9919948cdcd18b027/langchain_text_splitters-0.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for langsmith<0.2.0,>=0.1.17 from https://files.pythonhosted.org/packages/f8/53/0a22394aa520176b1981e9b7f02090425731b575e9ae28f86a7f5341208c/langsmith-0.1.143-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic<3.0.0,>=2.7.4 from https://files.pythonhosted.org/pac

In [2]:
pip install -U langchain-community

  Obtaining dependency information for langchain-community from https://files.pythonhosted.org/packages/cc/19/f8af1cdefe326730ae02bd653f7382693153baf0bac7a69537d7811cad5f/langchain_community-0.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for httpx-sse<0.5.0,>=0.4.0 from https://files.pythonhosted.org/packages/e1/9b/a181f281f65d776426002f330c31849b86b31fc9d848db62e16f03ff739f/httpx_sse-0.4.0-py3-none-any.whl.metadata
  Obtaining dependency information for pydantic-settings<3.0.0,>=2.4.0 from https://files.pythonhosted.org/packages/5e/f9/ff95fd7d760af42f647ea87f9b8a383d891cdb5e5dbd4613edaeb094252a/pydantic_settings-2.6.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 8.3 MB/s eta 0:00:00a 0:00:01
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that th

In [10]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.chains import AnalyzeDocumentChain
from langchain.agents import create_openai_functions_agent
from langchain.prompts import PromptTemplate

In [11]:
from langchain.document_loaders import PyMuPDFLoader

# Load PDF
loader = PyMuPDFLoader("10-Q4-2024-As-Filed.pdf")
documents = loader.load()

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv('key.env')  

key_string = os.getenv('open_ai_API_Key')

In [25]:
def parse_financial_section(result, section):
    section_data = []
    # Look for key numbers like Revenues, Total Assets, Cash Flows, etc., based on the section
    # Example: Using regex to capture key-value pairs like "Revenue: 100,000"
    patterns = [
        r"(Revenue|Expenses|Net Income|Assets|Liabilities|Equity|Cash Flow|Shareholders' Equity):?[\s]*([\d,\.]+)",
        # Add more patterns to capture other relevant financial terms and values
    ]
    for pattern in patterns:
        matches = re.findall(pattern, result)
        for match in matches:
            item, value = match
            section_data.append([section, item.strip(), value.strip()])
    return section_data

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
import re

# Define a prompt template
prompt_template = """
Extract the detailed financial data from the section "Financial Statements and Supplementary Data" of 10-k document:
1. Income Statement 
2. Balance Sheet 
3. Statement of Cash Flow 
4. Shareholders' Equity 
5. Other Financial Statements

Please make sure to only extract data relevant to the "Financial Statements and Supplementary Data" section and return the data in CSV format, with appropriate headers for each category.

Document Text: {text}


"""

prompt = PromptTemplate(input_variables=["text"], template=prompt_template)

# Initialize the language model
llm = OpenAI(api_key = key_string,temperature=0)

# Set up the LLM chain
llm_chain = LLMChain(prompt=prompt, llm=llm)

extracted_data = {
     "Income Statement":[],
     "Statement of Cash Flow":[],
     "Balance Sheet":[],
     "SHAREHOLDERS EQUITY":[],
     "other statements":[]
}
for document in documents:
    if "Financial Statements and Supplementary Data" in document.page_content:
        result = llm_chain.run({"text": document.page_content})

        # Use regex or string methods to check for and extract specific financial data
        # Extract Income Statement (based on the model's output format)
        if "Income Statement" in result:
            extracted_data["Income Statement"].extend(parse_financial_section(result, "Income Statement"))
        
        if "Statement of Cash Flow" in result:
            extracted_data["Statement of Cash Flow"].extend(parse_financial_section(result, "Statement of Cash Flow"))
        
        if "Balance Sheet" in result:
            extracted_data["Balance Sheet"].extend(parse_financial_section(result, "Balance Sheet"))
        
        if "SHAREHOLDERS EQUITY" in result:
            extracted_data["SHAREHOLDERS EQUITY"].extend(parse_financial_section(result, "SHAREHOLDERS EQUITY"))
        
        if "Other statements" in result:
            extracted_data["Other statements"].extend(parse_financial_section(result, "Other statements"))



# Print the extracted data in a readable format
for category, data in extracted_data.items():
    if data:  # Only print non-empty data
        print(f"\n### {category} ###")
        for item in data:
            print(f"Section: {item[0]}, Item: {item[1]}, Value: {item[2]}")    
# folder_path = "financial_data_folder"
# os.makedirs(folder_path, exist_ok=True)

# for category, data in extracted_data.items():
#     if data:  # Only save non-empty data
#         csv_filename = os.path.join(folder_path, f"{category.replace(' ', '_')}.csv")
        
#         # Write the CSV content to the file
#         with open(csv_filename, mode="w", encoding="utf-8") as file:
#             file.write(data)

#         print(f"Data for {category} saved to {csv_filename}")


### Income Statement ###
Section: Income Statement, Item: Net Income, Value: ,
Section: Income Statement, Item: Assets, Value: ,
Section: Income Statement, Item: Assets, Value: ,
Section: Income Statement, Item: Liabilities, Value: ,
Section: Income Statement, Item: Net Income, Value: ,
Section: Income Statement, Item: Assets, Value: ,
Section: Income Statement, Item: Assets, Value: ,

### Statement of Cash Flow ###
Section: Statement of Cash Flow, Item: Net Income, Value: ,
Section: Statement of Cash Flow, Item: Assets, Value: ,
Section: Statement of Cash Flow, Item: Assets, Value: ,
Section: Statement of Cash Flow, Item: Liabilities, Value: ,
Section: Statement of Cash Flow, Item: Net Income, Value: ,
Section: Statement of Cash Flow, Item: Assets, Value: ,
Section: Statement of Cash Flow, Item: Assets, Value: ,

### Balance Sheet ###
Section: Balance Sheet, Item: Net Income, Value: ,
Section: Balance Sheet, Item: Assets, Value: ,
Section: Balance Sheet, Item: Assets, Value: ,
Sectio